In [33]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data.sampler import SubsetRandomSampler

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, confusion_matrix
import seaborn as sn
import numpy as np
from numpy.random import seed
import matplotlib
import matplotlib.pyplot as plt
import pickle
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [3]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

seaborn    : 0.13.2
numpy      : 1.23.5
torchvision: 0.16.2
matplotlib : 3.5.2
torch      : 2.2.1



In [4]:
# Verifica a disponibilidade do uso da GPU (RTX 3060 Ti) para o treinamento do modelo
#!nvidia-smi
if torch.cuda.is_available():
    print(f"GPU atual: {torch.cuda.get_device_name(0)}")
else:
    print("Sem GPU disponível!")

GPU atual: NVIDIA GeForce RTX 3060 Ti


In [5]:
torch.manual_seed(8499)
seed(4899)
transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                               transforms.RandomRotation(10),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
                               transforms.Resize([200,200])]) #128,128

In [6]:
# Dados de Treino
train_data = ImageFolder(root = '/home/ulr1c8/CUDA_related/Mestrado/Brain_Tumor_MRI_figshare+SARTAJ+Br35H/Training', 
                         transform=transform)
print(train_data)

Dataset ImageFolder
    Number of datapoints: 5712
    Root location: /home/ulr1c8/CUDA_related/Mestrado/Brain_Tumor_MRI_figshare+SARTAJ+Br35H/Training
    StandardTransform
Transform: Compose(
               RandomHorizontalFlip(p=0.5)
               RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
               Resize(size=[200, 200], interpolation=bilinear, max_size=None, antialias=True)
           )


In [7]:
# Dados de Teste
test_data = ImageFolder(root = '/home/ulr1c8/CUDA_related/Mestrado/Brain_Tumor_MRI_figshare+SARTAJ+Br35H/Testing',
                       transform=transform)
print(test_data)

Dataset ImageFolder
    Number of datapoints: 1311
    Root location: /home/ulr1c8/CUDA_related/Mestrado/Brain_Tumor_MRI_figshare+SARTAJ+Br35H/Testing
    StandardTransform
Transform: Compose(
               RandomHorizontalFlip(p=0.5)
               RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
               Resize(size=[200, 200], interpolation=bilinear, max_size=None, antialias=True)
           )


In [8]:
# Número de amostras de treino
num_amostras_treino = len(train_data)
num_amostras_treino

5712

In [9]:
# Criamos um índice e o tornamos randômico
torch.manual_seed(8499)
seed(4899)
indices = list(range(num_amostras_treino))
np.random.shuffle(indices)

In [10]:
# Percentual dos dados de treino que usaremos no dataset de validação
valid_size = 0.25

In [11]:
# Agora fazemos o split para os dados de treino e validação
split = int(np.floor(valid_size * num_amostras_treino))
idx_treino, idx_valid = indices[split:], indices[:split]

In [12]:
# Definimos as amostras de treino
amostras_treino = SubsetRandomSampler(idx_treino)

In [13]:
# Definimos as amostras de validação
amostras_valid = SubsetRandomSampler(idx_valid)

In [14]:
# Número de subprocessos para carregar os dados
num_workers = 0
batch_size = 23

In [15]:
# Train Loader
torch.manual_seed(8499)
seed(4899)
train_loader = torch.utils.data.DataLoader(train_data, 
                                            batch_size = 28, 
                                            sampler = amostras_treino, 
                                            num_workers = num_workers)

In [16]:
# Data Loader de dados de validação
torch.manual_seed(8499)
seed(4899)
valid_loader = torch.utils.data.DataLoader(train_data, 
                                           batch_size = 28, 
                                           sampler = amostras_valid, 
                                           num_workers = num_workers)

In [17]:
# Test Loader
torch.manual_seed(8499)
seed(4899)
test_loader = torch.utils.data.DataLoader(test_data,
                                         batch_size=batch_size,
                                         shuffle = True)

In [18]:
# Arquitetura do Modelo
class ModeloCNN(nn.Module):
    
    # Método construtor
    def __init__(self):
        super(ModeloCNN, self).__init__()
        
        # Camada Convolucional de entrada 
        self.conv1 = nn.Conv2d(3, 16, 4, padding = 1, stride = 2)
        
        # Camada MaxPooling 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=4, stride=2, padding=1)
        
        # Camada de Dropout 1 (Regularização)
        self.dropout = nn.Dropout(0.3)
        
        # Camada Convolucional oculta 
        self.conv2 = nn.Conv2d(16, 32, 5, padding = 1)
        
        # Camada MaxPooling 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=5, stride=1, padding=0)
        
        # Camada de Dropout 2 (Regularização)
        self.dropout = nn.Dropout(0.3)
        
        # Camada Convolucional oculta 
        self.conv3 = nn.Conv2d(32, 64, 4, padding = 1, stride = 2)
        
        # Camada MaxPooling 3
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=1, padding=0)
        
        # Camada de Dropout 4 (Regularização)
        self.dropout = nn.Dropout(0.3)
        
        # Camada Totalmente Conectada 1
        self.fc1 = nn.Linear(20*20*64, 1000)
        
        # Camada totalmente Conectada 2
        self.fc2 = nn.Linear(1000,500)
        
        # Camada Totalmente Conectada 3
        self.fc3 = nn.Linear(500, 4)
    
    # Método Forward
    def forward(self, x):
        
        # Adiciona uma camada de ativação Relu para cada camada convolucional
        x = self.dropout(self.maxpool1(F.relu(self.conv1(x))))
        x = self.dropout(self.maxpool2(F.relu(self.conv2(x))))
        x = self.dropout(self.maxpool3(F.relu(self.conv3(x))))
        
        # Faz o "achatamento" da matriz resultante da convolução e cria um vetor
        x = x.view(x.size(0), 20*20*64)
        
        # Adiciona uma camada de dropout para regularização
        x = self.dropout(x)
        
        # Adiciona a 1ª camada oculta, com função de ativação relu
        x = F.relu(self.fc1(x))
        
        # Adiciona a 2ª camada oculta
        x = self.fc2(x)
        
        # Adiciona a 3ª camada oculta (classificação feita pelo modelo)
        x = self.fc3(x)
        
        return x

In [19]:
# Cria o modelo
modelo = ModeloCNN()
print(modelo)

ModeloCNN(
  (conv1): Conv2d(3, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (maxpool1): MaxPool2d(kernel_size=4, stride=2, padding=1, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.3, inplace=False)
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (maxpool2): MaxPool2d(kernel_size=5, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (maxpool3): MaxPool2d(kernel_size=3, stride=1, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=25600, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=500, bias=True)
  (fc3): Linear(in_features=500, out_features=4, bias=True)
)


In [20]:
# Movemos o modelo para a GPU se disponível
if torch.cuda.is_available():
    modelo.to("cuda")

In [21]:
# Função perda como categorical cross-entropy
criterion = nn.CrossEntropyLoss()

In [22]:
# Hiperparâmetro taxa de aprendizado
lr = 0.0001

In [23]:
# Otimizador
optimizer = optim.Adam(modelo.parameters(), lr = lr)

## Treinamento

In [24]:
# Número de épocas para treinar o modelo
num_epochs = 100

In [25]:
# Hiperparametro para controlar a mudança do erro em validação
erro_valid_min = np.Inf

In [26]:
torch.cuda.is_available()

True

### Treinamento do modelo (a execução pode demorar)

In [69]:
%%time

torch.manual_seed(8499)
seed(4899)

for epoch in range(1, num_epochs + 1):

    # Parâmetros para acompanhar o erro total em treinamento e validação
    erro_treino = 0.0
    erro_valid = 0.0
    
    # Inicia o treinamento do modelo
    modelo.train().to("cuda")
    
    # Loop pelos batches de dados de treino
    for batch_idx, (data, target) in enumerate(train_loader):
        
        # Move os tensores para a GPU se disponível
        if torch.cuda.is_available():
            data, target = data.to("cuda"), target.to("cuda")
        
        # Limpa os gradientes de todas as variáveis otimizadas
        optimizer.zero_grad()
        
        # Forward: calcula as saídas previstas
        output = modelo(data)
        
        # Calcula o erro no batch
        loss = criterion(output, target)
        
        # Backward: calcula o gradiente da perda em relação aos parâmetros do modelo
        loss.backward()
        
        # Realiza uma única etapa de otimização (atualização dos parâmetros)
        optimizer.step()
        
        # Atualiza o erro total em treino
        erro_treino += loss.item() * data.size(0)
        
    # Inicia a validação do modelo
    modelo.eval().cuda
    
    # Loop pelos batches de dados de validação
    for batch_idx, (data, target) in enumerate(valid_loader):
        
        # Move os tensores para a GPU se disponível
        if torch.cuda.is_available():
            data, target = data.to("cuda"), target.to("cuda")
        
        # Forward: calcula as saídas previstas
        output = modelo(data)
        
        # Calcula o erro no batch
        loss = criterion(output, target)
        
        # Atualiza o erro total de validação
        erro_valid += loss.item() * data.size(0)
    
    # Calcula o erro médio
    erro_treino = erro_treino / len(train_loader.dataset)
    erro_valid = erro_valid / len(valid_loader.dataset)
        
    # Print
    print('\nEpoch: {} \tErro em Treinamento: {:.6f} \tErro em Validação: {:.6f}'.format(epoch, 
                                                                                         erro_treino, 
                                                                                         erro_valid))
    
    # Salva o modelo sempre que a perda em validação diminuir
    if erro_valid <= erro_valid_min:
        print('Erro em Validação foi Reduzido ({:.6f} --> {:.6f}). Salvando o modelo...'.format(erro_valid_min,
                                                                                                 erro_valid))
        #pickle.dump(modelo, open('modelo_torch2.pkl', 'wb'))
        erro_valid_min = erro_valid


Epoch: 1 	Erro em Treinamento: 0.802672 	Erro em Validação: 0.230151
Erro em Validação foi Reduzido (inf --> 0.230151). Salvando o modelo...

Epoch: 2 	Erro em Treinamento: 0.533509 	Erro em Validação: 0.197814
Erro em Validação foi Reduzido (0.230151 --> 0.197814). Salvando o modelo...

Epoch: 3 	Erro em Treinamento: 0.467951 	Erro em Validação: 0.192418
Erro em Validação foi Reduzido (0.197814 --> 0.192418). Salvando o modelo...

Epoch: 4 	Erro em Treinamento: 0.410031 	Erro em Validação: 0.167634
Erro em Validação foi Reduzido (0.192418 --> 0.167634). Salvando o modelo...

Epoch: 5 	Erro em Treinamento: 0.377079 	Erro em Validação: 0.167910

Epoch: 6 	Erro em Treinamento: 0.351820 	Erro em Validação: 0.139635
Erro em Validação foi Reduzido (0.167634 --> 0.139635). Salvando o modelo...

Epoch: 7 	Erro em Treinamento: 0.324680 	Erro em Validação: 0.148075

Epoch: 8 	Erro em Treinamento: 0.307797 	Erro em Validação: 0.136795
Erro em Validação foi Reduzido (0.139635 --> 0.136795). Salv


Epoch: 85 	Erro em Treinamento: 0.010578 	Erro em Validação: 0.038004

Epoch: 86 	Erro em Treinamento: 0.007743 	Erro em Validação: 0.043926

Epoch: 87 	Erro em Treinamento: 0.010211 	Erro em Validação: 0.039407

Epoch: 88 	Erro em Treinamento: 0.018182 	Erro em Validação: 0.034590

Epoch: 89 	Erro em Treinamento: 0.011389 	Erro em Validação: 0.042993

Epoch: 90 	Erro em Treinamento: 0.011007 	Erro em Validação: 0.042973

Epoch: 91 	Erro em Treinamento: 0.008699 	Erro em Validação: 0.041093

Epoch: 92 	Erro em Treinamento: 0.013986 	Erro em Validação: 0.042917

Epoch: 93 	Erro em Treinamento: 0.010560 	Erro em Validação: 0.033715

Epoch: 94 	Erro em Treinamento: 0.015212 	Erro em Validação: 0.049924

Epoch: 95 	Erro em Treinamento: 0.012855 	Erro em Validação: 0.044194

Epoch: 96 	Erro em Treinamento: 0.011801 	Erro em Validação: 0.052602

Epoch: 97 	Erro em Treinamento: 0.011188 	Erro em Validação: 0.029394
Erro em Validação foi Reduzido (0.029628 --> 0.029394). Salvando o modelo...


In [27]:
# Carregando o modelo com menor erro de validação após treinamanto
modelo = pickle.load(open('modelo_torch2.pkl', 'rb'))

In [79]:
# Só usar caso a acurácia do modelo testado seja maior que a acurácia atual do arquivo modelo_torch.pkl
#pickle.dump(modelo, open('modelo_torch.pkl', 'wb'))

In [72]:
# Carregando o modelo com menor erro de validação geral, em arquivo salvo modelo_torch.pkl
modelo_atual = pickle.load(open('modelo_torch.pkl', 'rb'))
print(modelo_atual)

ModeloCNN(
  (conv1): Conv2d(3, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (maxpool1): MaxPool2d(kernel_size=4, stride=2, padding=1, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.3, inplace=False)
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (maxpool2): MaxPool2d(kernel_size=5, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (maxpool3): MaxPool2d(kernel_size=3, stride=1, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=25600, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=500, bias=True)
  (fc3): Linear(in_features=500, out_features=4, bias=True)
)


### Testando e Avaliando o Modelo Final

In [28]:
# Lista de classes das imagens
classes = ['glioma', 'meningioma', 'notumor', 'pituitary']

In [29]:
# Erro em teste
erro_teste = 0.0

In [30]:
# Controle de acertos do modelo
class_correct = list(0. for i in range(4))
class_total = list(0. for i in range(4))

In [31]:
%%time

torch.manual_seed(8499)
seed(4899)

prediction = torch.Tensor()
prediction = prediction.cuda()
y_test = torch.Tensor()
y_test = y_test.cuda()

torch.manual_seed(8499)
seed(8499)

# Lista de classes das imagens
classes = ['glioma', 'meningioma', 'notumor', 'pituitary']

# Erro em teste
erro_teste = 0.0

# Controle de acertos do modelo
class_correct = list(0. for i in range(4))
class_total = list(0. for i in range(4))

# Inicia a avaliação do modelo
modelo.eval()

# Loop pelos batches de dados de teste
for batch_idx, (data, target) in enumerate(test_loader):
    
    # Move os tensores para GPU se disponível
    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
        y_test = torch.cat((y_test, target), 0)
    
    # Forward
    output = modelo(data)
    
    # Calcula o erro
    loss = criterion(output, target)
    
    # Atualiza o erro em teste
    erro_teste += loss.item() * data.size(0)
    
    # Converte probabilidades de saída em classe prevista
    _, pred = torch.max(output, 1)
    prediction = torch.cat((prediction, pred), 0)
    
    # Compara as previsões com o rótulo verdadeiro
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not torch.cuda.is_available() else np.squeeze(correct_tensor.cpu().numpy())
    
    # Calcula a precisão do teste para cada classe
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# Erro médio em teste
erro_teste = erro_teste / len(test_loader.dataset)
print('\nErro em Teste: {:.6f}\n'.format(erro_teste))

# Calcula a acurácia para cada classe
for i in range(4):
    if class_total[i] > 0:
        print('Acurácia em Teste da classe %5s: %2d%% (%2d/%2d)' % (classes[i], 
                                                             100 * class_correct[i] / class_total[i],
                                                             np.sum(class_correct[i]), 
                                                             np.sum(class_total[i])))
    else:
        print('Acurácia em Teste de %5s:)' % (classes[i]))

# Calcula a acurácia total
print('\nAcurácia em Teste (Total): %.2f%% (%2d/%2d)' % (100. * np.sum(class_correct) / np.sum(class_total),
                                                        np.sum(class_correct), 
                                                        np.sum(class_total)))

y_test = y_test.cpu()
prediction = prediction.cpu()
cf_matrix = confusion_matrix(y_test, prediction)
print("\nMatriz de Confusão do Modelo com Camadas de Attention:")
print(cf_matrix)
print("\n")


Erro em Teste: 0.108381

Acurácia em Teste da classe glioma: 94% (284/300)
Acurácia em Teste da classe meningioma: 93% (285/306)
Acurácia em Teste da classe notumor: 99% (404/405)
Acurácia em Teste da classe pituitary: 99% (299/300)

Acurácia em Teste (Total): 97.03% (1272/1311)

Matriz de Confusão do Modelo com Camadas de Attention:
[[284  13   1   2]
 [  9 285   7   5]
 [  0   1 404   0]
 [  0   1   0 299]]


CPU times: user 22.8 s, sys: 307 ms, total: 23.1 s
Wall time: 4.44 s


In [37]:
print("Métricas de performance do modelo com camada de attention:")
print("Accuracy: ", accuracy_score(y_test, prediction))
print("Precision: ",precision_score(y_test, prediction, average='macro'))
print("F1-Score: ",f1_score(y_test, prediction, average='macro'))
print("Recall: ",recall_score(y_test, prediction, average='macro'))

Métricas de performance do modelo com camada de attention:
Accuracy:  0.9702517162471396
Precision:  0.9692474959322231
F1-Score:  0.9685545778452467
Recall:  0.9680591866376179


# Melhor modelo até agora (modelo_atual - modelo_torch)

In [76]:
# Arquitetura do Modelo
class ModeloCNN(nn.Module):
    
    # Método construtor
    def __init__(self):
        super(ModeloCNN, self).__init__()
        
        # Camada Convolucional de entrada 
        self.conv1 = nn.Conv2d(3, 16, 4, padding = 1, stride = 2)
        
        # Camada MaxPooling 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=4, stride=2, padding=1)
        
        # Camada de Dropout 1 (Regularização)
        self.dropout = nn.Dropout(0.3)
        
        # Camada Convolucional oculta 
        self.conv2 = nn.Conv2d(16, 32, 5, padding = 1)
        
        # Camada MaxPooling 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=5, stride=1, padding=0)
        
        # Camada de Dropout 2 (Regularização)
        self.dropout = nn.Dropout(0.3)
        
        # Camada Convolucional oculta 
        self.conv3 = nn.Conv2d(32, 64, 4, padding = 1, stride = 2)
        
        # Camada MaxPooling 3
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=1, padding=0)
        
        # Camada de Dropout 4 (Regularização)
        self.dropout = nn.Dropout(0.3)
        
        # Camada Totalmente Conectada 1
        self.fc1 = nn.Linear(20*20*64, 1000)
        
        # Camada totalmente Conectada 2
        self.fc2 = nn.Linear(1000,500)
        
        # Camada Totalmente Conectada 3
        self.fc3 = nn.Linear(500, 4)
    
    # Método Forward
    def forward(self, x):
        
        # Adiciona uma camada de ativação Relu para cada camada convolucional
        x = self.dropout(self.maxpool1(F.relu(self.conv1(x))))
        x = self.dropout(self.maxpool2(F.relu(self.conv2(x))))
        x = self.dropout(self.maxpool3(F.relu(self.conv3(x))))
        
        # Faz o "achatamento" da matriz resultante da convolução e cria um vetor
        x = x.view(x.size(0), 20*20*64)
        
        # Adiciona uma camada de dropout para regularização
        x = self.dropout(x)
        
        # Adiciona a 1ª camada oculta, com função de ativação relu
        x = F.relu(self.fc1(x))
        
        # Adiciona a 2ª camada oculta
        x = self.fc2(x)
        
        # Adiciona a 3ª camada oculta (classificação feita pelo modelo)
        x = self.fc3(x)
        
        return x

In [77]:
# Carregando o modelo com menor erro de validação geral, em arquivo salvo modelo_torch.pkl
modelo_atual = pickle.load(open('modelo_torch.pkl', 'rb'))
print(modelo_atual)

ModeloCNN(
  (conv1): Conv2d(3, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (maxpool1): MaxPool2d(kernel_size=4, stride=2, padding=1, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.3, inplace=False)
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (maxpool2): MaxPool2d(kernel_size=5, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (maxpool3): MaxPool2d(kernel_size=3, stride=1, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=25600, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=500, bias=True)
  (fc3): Linear(in_features=500, out_features=4, bias=True)
)


In [78]:
%%time

torch.manual_seed(8499)
seed(4899)

prediction = torch.Tensor()
prediction = prediction.cuda()
y_test = torch.Tensor()
y_test = y_test.cuda()

torch.manual_seed(8499)
seed(8499)

# Lista de classes das imagens
classes = ['glioma', 'meningioma', 'notumor', 'pituitary']

# Erro em teste
erro_teste = 0.0

# Controle de acertos do modelo
class_correct = list(0. for i in range(4))
class_total = list(0. for i in range(4))

# Inicia a avaliação do modelo
modelo_atual.eval()

# Loop pelos batches de dados de teste
for batch_idx, (data, target) in enumerate(test_loader):
    
    # Move os tensores para GPU se disponível
    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
        y_test = torch.cat((y_test, target), 0)
    
    # Forward
    output = modelo_atual(data)
    
    # Calcula o erro
    loss = criterion(output, target)
    
    # Atualiza o erro em teste
    erro_teste += loss.item() * data.size(0)
    
    # Converte probabilidades de saída em classe prevista
    _, pred = torch.max(output, 1)
    prediction = torch.cat((prediction, pred), 0)
    
    # Compara as previsões com o rótulo verdadeiro
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not torch.cuda.is_available() else np.squeeze(correct_tensor.cpu().numpy())
    
    # Calcula a precisão do teste para cada classe
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# Erro médio em teste
erro_teste = erro_teste / len(test_loader.dataset)
print('\nErro em Teste: {:.6f}\n'.format(erro_teste))

# Calcula a acurácia para cada classe
for i in range(4):
    if class_total[i] > 0:
        print('Acurácia em Teste da classe %5s: %2d%% (%2d/%2d)' % (classes[i], 
                                                             100 * class_correct[i] / class_total[i],
                                                             np.sum(class_correct[i]), 
                                                             np.sum(class_total[i])))
    else:
        print('Acurácia em Teste de %5s:)' % (classes[i]))

# Calcula a acurácia total
print('\nAcurácia em Teste (Total): %.2f%% (%2d/%2d)' % (100. * np.sum(class_correct) / np.sum(class_total),
                                                        np.sum(class_correct), 
                                                        np.sum(class_total)))
y_test = y_test.cpu()
prediction = prediction.cpu()
cf_matrix = confusion_matrix(y_test, prediction)
print("\nMatriz de Confusão do Modelo com Camadas de Attention:")
print(cf_matrix)
print("\n")


Erro em Teste: 0.121649

Acurácia em Teste da classe glioma: 93% (280/300)
Acurácia em Teste da classe meningioma: 93% (286/306)
Acurácia em Teste da classe notumor: 100% (405/405)
Acurácia em Teste da classe pituitary: 99% (298/300)

Acurácia em Teste (Total): 96.80% (1269/1311)

Matriz de Confusão do Modelo com Camadas de Attention:
[[280  18   0   2]
 [  7 286   6   7]
 [  0   0 405   0]
 [  0   2   0 298]]


CPU times: user 22.1 s, sys: 110 ms, total: 22.3 s
Wall time: 3.42 s
